In [1]:
import numpy as np
import matplotlib.pyplot as plt
from image_graph_util import img_conversion_v7
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from emnist import extract_training_samples,extract_test_samples
from emnist import list_datasets
from sklearn.utils import shuffle
import cv2
import glob

2023-07-25 14:33:16.102091: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-07-25 14:33:16.102116: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
img=img_conversion_v7.image_convert()


patch=2


In [3]:
ds, info = tfds.load('malaria', split='train', shuffle_files=True, with_info=True)

2023-07-25 14:33:53.152240: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/user/home/sahaj432/.local/lib/python3.9/site-packages/cv2/../../lib64:/cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-07-25 14:33:53.152274: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-25 14:33:53.152294: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0193): /proc/driver/nvidia/version does not exist
2023-07-25 14:33:53.152627: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other opera

In [4]:
train_images = []
train_labels = []

for example in ds:
    train_images.append(example['image'].numpy())
    train_labels.append(example['label'].numpy())

In [5]:
print(len(train_images))
print(len(train_labels))

27558
27558


In [6]:
images=np.array([cv2.resize(i,(100,100), interpolation= cv2.INTER_CUBIC) for i in train_images])


In [7]:
images = [i/255 for i in images]
labels = np.array(train_labels)

In [8]:
images=np.array(images,dtype="float16")

In [9]:
images.shape

(27558, 100, 100, 3)

In [10]:
train,test,train_lab,test_lab=train_test_split(images,labels,test_size=0.30,random_state=42)


In [11]:
print(train.shape)
print(test.shape)
print(train_lab.shape)
print(test_lab.shape)

(19290, 100, 100, 3)
(8268, 100, 100, 3)
(19290,)
(8268,)


In [12]:
regular_train=train
reg_train_label=train_lab

In [13]:
regular_val,regular_test,reg_val_label,reg_test_label=train_test_split(test,test_lab,test_size=0.50,random_state=42)


In [14]:
print(regular_train.shape)
print(regular_val.shape)
print(regular_test.shape)


(19290, 100, 100, 3)
(4134, 100, 100, 3)
(4134, 100, 100, 3)


In [15]:
from collections import Counter
#print((Counter(y_val)))
print(np.sort(list(Counter(reg_train_label).keys())))
classes=len(np.sort(list(Counter(reg_train_label).keys())))

[0 1]


In [16]:
reg_train_patch=img.to_patcharray(regular_train[:],patch_size=(patch,patch))
reg_val_patch=img.to_patcharray(regular_val[:],patch_size=(patch,patch))
reg_test_patch=img.to_patcharray(regular_test[:],patch_size=(patch,patch))



In [17]:
print(reg_train_patch.shape)
print(reg_val_patch.shape)
print(reg_test_patch.shape)

(19290, 3, 2500, 2, 2)
(4134, 3, 2500, 2, 2)
(4134, 3, 2500, 2, 2)


In [18]:
import networkx as nx


def compute_laplacian_channel(df_in):
    adj_ary=np.ones(reg_train_patch.shape[-2]*reg_train_patch.shape[-1])-np.eye(reg_train_patch.shape[-2]*reg_train_patch.shape[-1])

    edge=np.array(np.where(adj_ary==1))
    ed0=edge[0]
    ed1=edge[1]

    no_img,no_ch, no_patch,patch_x,patch_y=df_in.shape
    temp=[]
    for i in range(no_img):
        i_ary = np.eye(patch_x*patch_y)
        i_ary4d=i_ary[np.newaxis,np.newaxis,:,:]
        weighted_adj=np.zeros((no_ch, no_patch,patch_x*patch_y,patch_x*patch_y))
        df=df_in[i].reshape(no_ch,no_patch,-1)
        pixel1=df[:,:,ed0]
        pixel2=df[:,:,ed1]  
        edu_distance=np.sqrt((pixel1-pixel2)**2)+ 10**(-8)
        weighted_adj[:,:,ed0,ed1]=edu_distance
        sum_r=np.sum(weighted_adj,axis=2)
        temp_adj=sum_r[:,:,:,np.newaxis]
        deg=temp_adj*i_ary4d
        L=deg-weighted_adj
        temp.append(L)
    return temp



In [19]:
from multiprocessing import Pool
import multiprocessing



num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)


Number of CPU cores: 48


In [20]:
train_patch1=np.array_split(reg_train_patch[:], num_cores)
val_patch1=np.array_split(reg_val_patch[:], num_cores)
test_patch1=np.array_split(reg_test_patch[:], num_cores)

In [21]:
print(len(train_patch1))
print(len(val_patch1))
print(len(test_patch1))

48
48
48


In [22]:
num_matrices, num_channels, num_patches, height, width = train_patch1[0].shape

In [23]:
with Pool(processes=num_cores) as pool:
        # Define a list of numbers
    train_p = train_patch1
    
        # Apply the square function to each number using the pool
    g_train=pool.map(compute_laplacian_channel,train_p)
pool.close()


    
        #results = pool.map(calculate_visibility, numbers)


In [24]:
temp1=[]
for submatrix in g_train:
    temp1.extend(submatrix)


In [25]:
gersh_train= np.array([np.asarray(submatrix) for submatrix in temp1])


In [26]:
print("d")

d


In [27]:
with Pool(processes=num_cores) as pool1:
        # Define a list of numbers
    val_p = val_patch1
    
        # Apply the square function to each number using the pool
    g_val=pool1.map(compute_laplacian_channel,val_patch1)    
pool1.close()



In [28]:
temp2=[]
for submatrix in g_val:
    temp2.extend(submatrix)


In [29]:
gersh_val= np.array([np.asarray(submatrix) for submatrix in temp2])


In [30]:
print("d")

d


In [31]:
with Pool(processes=num_cores) as pool2:
        # Define a list of numbers
    test_p = test_patch1
    
        # Apply the square function to each number using the pool
    g_test=pool2.map(compute_laplacian_channel,test_patch1) 
pool2.close()


In [32]:
temp3=[]
for submatrix in g_test:
    temp3.extend(submatrix)


In [33]:
gersh_test= np.array([np.asarray(submatrix) for submatrix in temp3])


gersh_train=np.array(img1.to_gridlaparray(train_patch))
gersh_val=np.array(img1.to_gridlaparray(val_patch))
gersh_test=np.array(img1.to_gridlaparray(test_patch))

In [34]:
print(gersh_train.shape)
print(gersh_val.shape)
print(gersh_test.shape)

(19290, 3, 2500, 4, 4)
(4134, 3, 2500, 4, 4)
(4134, 3, 2500, 4, 4)


In [35]:
lap_train=gersh_train
lap_val=gersh_val
lap_test=gersh_test

In [36]:
print(lap_train.shape)
print(lap_val.shape)
print(lap_test.shape)

(19290, 3, 2500, 4, 4)
(4134, 3, 2500, 4, 4)
(4134, 3, 2500, 4, 4)


In [37]:
rlap_train=np.moveaxis(lap_train,1,-1)
rlap_val=np.moveaxis(lap_val,1,-1)
rlap_test=np.moveaxis(lap_test,1,-1)



In [38]:
rlap_train=rlap_train.reshape(rlap_train.shape[0],rlap_train.shape[1]*rlap_train.shape[2],rlap_train.shape[3],rlap_train.shape[4])
rlap_val=rlap_val.reshape(rlap_val.shape[0],rlap_val.shape[1]*rlap_val.shape[2],rlap_val.shape[3],rlap_val.shape[4])
rlap_test=rlap_test.reshape(rlap_test.shape[0],rlap_test.shape[1]*rlap_test.shape[2],rlap_test.shape[3],rlap_test.shape[4])

In [39]:
print(rlap_train.shape)
print(rlap_train.shape)
print(rlap_train.shape)

(19290, 10000, 4, 3)
(19290, 10000, 4, 3)
(19290, 10000, 4, 3)


# Gershgorien 2D CNN

In [40]:
input_shape = (rlap_train.shape[1],rlap_train.shape[2],rlap_train.shape[3])
num_classes = classes
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Conv2D(filters= 32, kernel_size=(3,3), activation='relu',padding='same',input_shape= input_shape))
model1.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2),strides=None,padding='valid'))
model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),padding='same',activation='relu'))
model1.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 1),strides=None,padding='valid'))

model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),padding='same',activation='relu'))
model1.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 1),strides=None,padding='valid'))

model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(512, activation='relu'))
model1.add(tf.keras.layers.Dropout(0.1))
model1.add(tf.keras.layers.Dense(512, activation='relu'))
model1.add(tf.keras.layers.Dense(num_classes, activation='softmax'))


In [41]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 10000, 4, 32)      896       
                                                                 
 average_pooling2d (AverageP  (None, 5000, 2, 32)      0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 5000, 2, 32)       9248      
                                                                 
 average_pooling2d_1 (Averag  (None, 2500, 2, 32)      0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 2500, 2, 32)       9248      
                                                                 
 average_pooling2d_2 (Averag  (None, 1250, 2, 32)      0

In [42]:
model1.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [43]:
history_log=model1.fit(rlap_train, reg_train_label, epochs=10,verbose=1,validation_data=(rlap_val, reg_val_label))


Epoch 1/10
603/603 [==============================] - 179s 296ms/step - loss: 0.2519 - acc: 0.9004 - val_loss: 0.2155 - val_acc: 0.9347
Epoch 2/10
603/603 [==============================] - 174s 289ms/step - loss: 0.1498 - acc: 0.9538 - val_loss: 0.1608 - val_acc: 0.9458
Epoch 3/10
603/603 [==============================] - 173s 287ms/step - loss: 0.1018 - acc: 0.9659 - val_loss: 0.1717 - val_acc: 0.9461
Epoch 4/10
603/603 [==============================] - 172s 285ms/step - loss: 0.0517 - acc: 0.9829 - val_loss: 0.2396 - val_acc: 0.9422
Epoch 5/10
603/603 [==============================] - 172s 285ms/step - loss: 0.0279 - acc: 0.9908 - val_loss: 0.3068 - val_acc: 0.9434
Epoch 6/10
603/603 [==============================] - 171s 284ms/step - loss: 0.0175 - acc: 0.9950 - val_loss: 0.3339 - val_acc: 0.9439
Epoch 7/10
603/603 [==============================] - 187s 309ms/step - loss: 0.0149 - acc: 0.9960 - val_loss: 0.3597 - val_acc: 0.9448
Epoch 8/10
603/603 [============================

In [44]:
loss, accuracy = model1.evaluate(rlap_test, reg_test_label)
print(f' Model loss on the test set: {loss}')
print(f' Model accuracy on the test set: {100*accuracy}')

130/130 [==============================] - 5s 39ms/step - loss: 0.5593 - acc: 0.9393
 Model loss on the test set: 0.5593317747116089
 Model accuracy on the test set: 93.92839670181274


In [45]:
predictions = model1.predict(rlap_test)
pred_arg = np.argmax(predictions, axis=1)

In [46]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(reg_test_label,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[1940  139]
 [ 112 1943]]


In [47]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [48]:
print(accuracy_score(reg_test_label, pred_arg)*100)
print(f1_score(reg_test_label,pred_arg,average='macro')*100)
print(recall_score(reg_test_label, pred_arg, average="weighted")*100)
print(precision_score(reg_test_label, pred_arg, average="weighted")*100)

93.92839864537977
93.92839544792035
93.92839864537977
93.93635484880058
